In [1]:
import pandas as pd
import numpy as np
import functions as f
import networkx as nx
import matplotlib.pyplot as plt

import os

### Getting data from website (residentevildatabase.com)

In [2]:
if 'data' not in os.listdir("../"):
    df = f.get_all_personas()
    os.mkdir("../data")
    df.to_excel("../data/dataframe.xlsx")
else:
    df = pd.read_excel("../data/dataframe.xlsx")

### Transforming data to transacional dataset

In [3]:
df_filter = df[df["apparition"]!='Biohazard - Resident Evil: Operation Raccoon City']

df_analysis = pd.merge( left=df_filter, right=df_filter,
                        how="left", on="apparition",
                        suffixes=("_1", "_2") )

df_analysis = df_analysis[ df_analysis["persona_1"] != df_analysis["persona_2"] ]

df_analysis["names"] = df_analysis.apply(f.sort_names, axis=1)
df_analysis = df_analysis.drop_duplicates(subset=["names", "apparition"])

df_analysis = df_analysis[["persona_1", "persona_2", "apparition"]]

df_analysis["persona_1"] = df_analysis["persona_1"].apply( f.fix_persona )
df_analysis["persona_2"] = df_analysis["persona_2"].apply( f.fix_persona )

group = df_analysis.groupby(by=["persona_1", "persona_2"])

df_1 = group["apparition"].count().reset_index()
df_1.rename(columns={"apparition":"weight"}, inplace=True)

df_2 = group["apparition"].apply( lambda x: [i for i in x][0]  ).reset_index()
df_2.rename(columns={"apparition":"first_game"}, inplace=True)

df_3 = group["apparition"].apply( lambda x: [i for i in x][-1]  ).reset_index()
df_3.rename(columns={"apparition":"last_game"}, inplace=True)

df_analysis = pd.merge(left=df_1, right=df_2, how='inner')
df_analysis = pd.merge(left=df_analysis, right=df_3, how='inner')

df_analysis.head()

,persona_1,persona_2,weight,first_game,last_game
0,Ada Wong,Adam Benford,1,Biohazard - Resident Evil 6,Biohazard - Resident Evil 6
1,Ada Wong,Albert Wesker,1,Biohazard - Resident Evil 4,Biohazard - Resident Evil 4
2,Ada Wong,Alexander Ashford,1,Biohazard - Resident Evil: The Darkside Chroni...,Biohazard - Resident Evil: The Darkside Chroni...
3,Ada Wong,Alexander Buddy Kozachenko,1,Biohazard - Resident Evil: Damnation,Biohazard - Resident Evil: Damnation
4,Ada Wong,Alexia Ashford,1,Biohazard - Resident Evil: The Darkside Chroni...,Biohazard - Resident Evil: The Darkside Chroni...


In [4]:
G = nx.from_pandas_edgelist(df=df_analysis, source="persona_1", target="persona_2",
                            edge_attr=["persona_1", "persona_2", "first_game", "last_game", 'weight'],
                            create_using=nx.Graph())

f.plot_df(G)

Loading BokehJS ...

In [5]:
best_persons = ['Jill Valentine', 'Ada Wong', 'Leon S Kennedy', 'Claire Redfield', 'Chris Redfield', 'Albert Wesker']

f.plot_persona_network(df_analysis, best_persons[0])

Loading BokehJS ...